In [1]:
!pip install scrapy
!pip install driver
# requests will allow us to make a web request

In [38]:
import requests
# BeautifulSoup will allow us to easily parse the website's HTML code
from bs4 import BeautifulSoup
import re
import pandas as pd
import pandas as pd
# Import a scrapy Selector
from scrapy import Selector
import requests
import re
import pandas as pd
import driver
from scrapy import Selector

In [39]:
################ Get the team name and team_id for 2021-2022 season, if you want to change season, just change 
url = "https://stats.ncaa.org/selection_rankings/nitty_gritties/27203"
headers = {
"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54"
       }      
resp=requests.get(url,headers=headers)

html = requests.get( url,headers=headers ).content
# Create the Selector object sel from html
sel = Selector( text=html )
useful_info = sel.xpath('//html//body[@id="body"]//a[@target="TEAM_PAGE"and @class="skipMask"]')
useful_info
text = useful_info.xpath('./text()').extract()
href = useful_info.xpath('./@href').extract()
text
href
team = pd.DataFrame(text)
team=team.rename(columns = {0:"team_name"})
href = pd.DataFrame(href)
href=href.rename(columns = {0:"team_id"})
team["team_id"]=href

team["team_id"]=team['team_id'].map(lambda x: str(x)[7:])
team

,team_name,team_id
0,Villanova,532566
1,Virginia,532568
2,Kansas,532715
3,Texas Southern,532552
4,Toledo,532558
...,...,...
296,South Alabama,532525
297,Furman,532683
298,Ga. Southern,532687
299,Hartford,532693


In [ ]:
number = len(team.index)
df_pbp= pd.DataFrame(columns=["gameid","date","location","awayTeam","homeTeam","inn","topBottom","text","awayScore","homeScore"])

In [18]:
for t in range(number):
    team_idd = team.at[t,"team_id"]
    url2 = "https://stats.ncaa.org/teams/"+str(team_idd)+""
    print(url2)
    headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54"
       }      
    resp=requests.get(url2,headers=headers)
    html2 = requests.get( url2,headers=headers ).content
    # Create the Selector object sel from html
    sel2 = Selector( text=html2 )
    specific_info = sel2.xpath('//html//a[@target="BOX_SCORE_WINDOW"and @class="skipMask"]')
    specific_info

    majorteam_info = sel2.xpath('//html//div[@id="facility_div"]/div/@id').extract_first()
    majorteam_info
    href2 = specific_info.xpath('./@href').extract()
    href2

    href2 = pd.DataFrame(href2)
    href2=href2.rename(columns={0:"contest_id"})
    href2["team_id"]=majorteam_info
    href2["contest_id"]= href2['contest_id'].map(lambda x: str(x)[10:17])
    href2["team_id"]=href2['team_id'].map(lambda x: str(x)[12:])
    
    contest_info = href2
    contest_info
    
    n_c =len(contest_info)
    n_c
    
    for q in range(n_c):
        contest_idd = contest_info.at[q,"contest_id"]
        url3 = "https://stats.ncaa.org/contests/"+str(contest_idd)+"/box_score"
        headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54"
           }      
        resp=requests.get(url3,headers=headers)
    
        html3 = requests.get( url3,headers=headers ).content
        # Create the Selector object sel from html
        sel3 = Selector( text=html3 )
        specific_info = sel3.xpath('//html//ul[@class="level1"]//li[3]/a/@href').extract()
        specific_info=pd.DataFrame(specific_info).rename(columns = {0:"game_id"})        
        specific_info['game_id'] = specific_info['game_id'].apply(lambda x: x.split('/')[-1])
    
        m=specific_info.iloc[0,0]
        
    
    
        url = "https://stats.ncaa.org/game/play_by_play/"+str(m)+""
        header = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
        }
        source = requests.get(url, headers=header).text
        soup = BeautifulSoup(source, 'lxml')
        
        

        game_inn_scores=[{}]

        # Get the gameid...
        gameid = int(soup.find('ul',class_ = "level1",id = "root").select("li > a")[0]['href'].rsplit('/', 1)[-1])

        # Get the total number of innings in the game...
        num_innings = len(soup.find('table', class_ = 'mytable', width = "40%").tr.find_all())-4

        # Get the teamid of the AWAY team
        away_team = int(soup.find('a', class_="skipMask", target="TEAMS_WIN")['href'].rsplit('/', 1)[-1])
        # I'm definitely being a little sloppy here, but...
        # soup.find('a', class_="skipMask", target="TEAMS_WIN")['href']
        # ...is exactly equivalent to...
        # temp = soup.find('a', class_="skipMask", target="TEAMS_WIN")
        # temp['href']

        # Get the teamid of the HOME team
        home_team = int(soup.find_all('a', class_="skipMask", target="TEAMS_WIN")[-1]['href'].rsplit('/', 1)[-1])

        # Get the scores for the AWAY team
        away_inn_scores=soup.find('table', class_ = 'mytable', width = "40%").select('tr')[1].find_all('td')
        # Get the scores for the HOME team
        home_inn_scores=soup.find('table', class_ = 'mytable', width = "40%").select('tr')[2].find_all('td')

        # Capture the game notes (if there are any--and they may be multiple)
        # These may help us down the road to match pitchers with hitters in each at-bat
        notes = ''
        for tag in soup.find_all('table', align="center", width="50%")[0].find_all('td'):
            if not tag.text.startswith('Weather:') and not tag.text.startswith('Game:'):
                notes += tag.text + ";"

        # Get the game date
        game_date = soup.find('td', text=re.compile('Game Date:')).find_next('td').text.strip()

        # Get the game location
        game_location = soup.find('td', text=re.compile('Location:')).find_next('td').text.strip()

        # Get rid of team name item
        away_inn_scores.pop(0)
        home_inn_scores.pop(0)

        # Remove and store total errors for game
        a_e = away_inn_scores.pop()
        h_e = home_inn_scores.pop()

        # Remove and store total hits for game
        a_h = away_inn_scores.pop()
        h_h = home_inn_scores.pop()

        # Remove and store total runs for game
        a_r = away_inn_scores.pop()
        h_r = home_inn_scores.pop()

        # Loop through innings and store values properly
        for k in range(len(away_inn_scores)):
            game_inn_scores.append({'gameid' : gameid, 'teamid' : away_team, 'inn' : k + 1, 'runs_scored' : away_inn_scores[k].text})
            game_inn_scores.append({'gameid' : gameid, 'teamid' : home_team, 'inn' : k + 1, 'runs_scored' : home_inn_scores[k].text})

        # To create a dataframe: df = pd.DataFrame(game_inn_scores)
        
       
        # Scrape the play-by-play data
        pbp_recs=[{}]
        # Set the inning counter
        i=1
        # Loop through innings
        for inning in soup.find_all('table', class_ = 'mytable', width = "1000px"):
        # Loop through records in each inning, ignore tags with "grey_heading" class
            for tag in inning.find_all('tr', class_=lambda x: x != 'grey_heading')[2:]:
        # Check for the hyphen in the score, ignore "blank" rows without a score
                if tag.find(string=re.compile("-")):
                    score = tag.select('td:nth-of-type(2)')[0].string
        # Determine if data is for the top or bottom of the inning
                    if tag.select('td:nth-of-type(3)')[0].string is None:
                        pbp_recs.append({'gameid' : gameid, 'date' : game_date, 'location' : game_location, 'awayTeam' : away_team, 'homeTeam' : home_team, 'inn' : i, 'topBottom' : 'top', 'text' : tag.select('td:nth-of-type(1)')[0].string, 'awayScore' : score.split('-')[0].strip(), 'homeScore' : score.split('-')[1].strip()})
        #                print('Top ' + str(i) + '...' + tag.select('td:nth-of-type(1)')[0].string + ' ...Score ' + str(score))
                    else:
                        pbp_recs.append({'gameid' : gameid, 'date' : game_date, 'location' : game_location, 'awayTeam' : away_team, 'homeTeam' : home_team, 'inn' : i, 'topBottom' : 'bot', 'text' : tag.select('td:nth-of-type(3)')[0].string, 'awayScore' : score.split('-')[0].strip(), 'homeScore' : score.split('-')[1].strip()})
        #                print('Bottom ' + str(i) + '...' + tag.select('td:nth-of-type(3)')[0].string + ' ...Score ' + str(score))
            i+=1
        # To create a dataframe: df = pd.DataFrame(pbp_recs)
        df_pbp=df_pbp.append(pbp_recs)
        


https://stats.ncaa.org/teams/532566
https://stats.ncaa.org/teams/532568
https://stats.ncaa.org/teams/532715


KeyboardInterrupt: 

In [19]:
df_pbp.drop_duplicates(inplace=True) 
df_pbp.reset_index(drop=True, inplace=True)
df_pbp

,gameid,date,location,awayTeam,homeTeam,inn,topBottom,text,awayScore,homeScore
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5201718.0,02/11/2022,"Atlanta, Ga.",532566.0,532689.0,1.0,top,"Giampolo, A. struck out swinging.",0,0
2,5201718.0,02/11/2022,"Atlanta, Ga.",532566.0,532689.0,1.0,top,"Rauch, P. singled up the middle.",0,0
3,5201718.0,02/11/2022,"Atlanta, Ga.",532566.0,532689.0,1.0,top,"Smith, C. walked; Rauch, P. advanced to second.",0,0
4,5201718.0,02/11/2022,"Atlanta, Ga.",532566.0,532689.0,1.0,top,"Rauch, P. advanced to third.",0,0
...,...,...,...,...,...,...,...,...,...,...
10622,5241484.0,03/20/2022,"Lawrence, Kan.",532529.0,532715.0,7.0,top,"Bruno, Olivia to p.",1,6
10623,5241484.0,03/20/2022,"Lawrence, Kan.",532529.0,532715.0,7.0,top,"/ for Hamilton, Kasey.",1,6
10624,5241484.0,03/20/2022,"Lawrence, Kan.",532529.0,532715.0,7.0,top,Rylee Nicholson struck out looking.,1,6
10625,5241484.0,03/20/2022,"Lawrence, Kan.",532529.0,532715.0,7.0,top,Jordyn Pender grounded out to 1b unassisted.,1,6


In [41]:
df_pbp.to_csv('pbp_data.csv',index = False)